Building a causal
graphical model

 Building a causal DAG to model a DGP
 Using your causal graph as a communication,
computation, and reasoning tool
 Building a causal DAG in pgmpy and Pyro
 Training a probabilistic machine learning model
using the causal DAG as a scaffold

In [1]:
def true_dgp(jenny_inclination, brian_inclination, window_strength):
    jenny_throws_rock = jenny_inclination > 0.5
    brian_throws_rock = brian_inclination > 0.5
    if jenny_throws_rock and brian_throws_rock:
        strength_of_impact = 0.8
    elif jenny_throws_rock or brian_throws_rock:
        strength_of_impact = 0.6
    else:
        strength_of_impact = 0.0
    window_breaks = strength_of_impact > window_strength
    return jenny_throws_rock, brian_throws_rock, window_breaks

In [18]:
from pgmpy.models import DiscreteBayesianNetwork

model = DiscreteBayesianNetwork(
    [("A", "E"), ("S", "E"), ("E", "O"), ("E", "R"), ("O", "T"), ("R", "T")]
)


In [19]:
import pandas as pd
url='https://raw.githubusercontent.com/altdeep/causalML/master/datasets/transportation_survey.csv'
data = pd.read_csv(url)
data

,A,S,E,O,R,T
0,adult,F,high,emp,small,train
1,young,M,high,emp,big,car
2,adult,M,uni,emp,big,other
3,old,F,uni,emp,big,car
4,young,F,uni,emp,big,car
...,...,...,...,...,...,...
495,young,M,high,emp,big,other
496,adult,M,high,emp,big,car
497,young,M,high,emp,small,train
498,young,M,high,emp,small,car


In [21]:
model.fit(data)
causal_markov_kernels = model.get_cpds()
print(causal_markov_kernels)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'A': 'C', 'S': 'C', 'E': 'C', 'O': 'C', 'R': 'C', 'T': 'C'}


[<TabularCPD representing P(A:3) at 0x12a87aa50>, <TabularCPD representing P(E:2 | A:3, S:2) at 0x12a87bad0>, <TabularCPD representing P(S:2) at 0x12a879850>, <TabularCPD representing P(O:2 | E:2) at 0x12a87a270>, <TabularCPD representing P(R:2 | E:2) at 0x12a878ef0>, <TabularCPD representing P(T:3 | O:2, R:2) at 0x12a87a3f0>]


In [22]:
cmk_T = causal_markov_kernels[-1]
print(cmk_T)

+----------+---------------------+-----+--------------------+----------+
| O        | O(emp)              | ... | O(self)            | O(self)  |
+----------+---------------------+-----+--------------------+----------+
| R        | R(big)              | ... | R(big)             | R(small) |
+----------+---------------------+-----+--------------------+----------+
| T(car)   | 0.7034313725490197  | ... | 0.4444444444444444 | 1.0      |
+----------+---------------------+-----+--------------------+----------+
| T(other) | 0.13480392156862744 | ... | 0.3333333333333333 | 0.0      |
+----------+---------------------+-----+--------------------+----------+
| T(train) | 0.16176470588235295 | ... | 0.2222222222222222 | 0.0      |
+----------+---------------------+-----+--------------------+----------+
